In [89]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering

In [6]:
with open('spark_dependency_results.json') as f:
    spark_results = json.load(f)

In [64]:
#spark_results['variables']

### main_dict is for Directory -> Value ###
### main_dict_val is for Value -> Directory ###


main_dict = {}
main_dict_val = {}
main_dict_counter = 0
dict_array = []
min_set = set()
for element in spark_results['variables']:
    #print(element)
    
    ### Replace this with the initial JSON directory ###
    element_tbc = element.replace('C:\\Users\\tanji\\Desktop\\', '')
    main_dict[element_tbc] = main_dict_counter
    main_dict_val[main_dict_counter] = element_tbc
    main_dict_counter += 1

In [48]:
#main_dict

In [53]:
rootdir = 'spark-master'
dir_arr = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        ### Only look for Java Files ###
        if ".java" in str(os.path.join(subdir, file)):
            #print(os.path.join(subdir, file))
            dir_arr.append(str(os.path.join(subdir, file)))
            
print(len(dir_arr))

1106


In [27]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(dir_arr)

true_k = 300
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

#print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
#for i in range(true_k):
    #print("Cluster %d:" % i),
    #for ind in order_centroids[i, :10]:
        #print(' %s' % terms[ind]),
    #print

#print("\n")
print("Prediction")

Y = vectorizer.transform(["spark-master\\common\\kvstore\\src\\main\\java\\org\\apache\\spark\\util\\kvstore\\ArrayWrappers.java"])
prediction = model.predict(Y)
print(prediction)

Y = vectorizer.transform(["spark-master\\sql\\hive-thriftserver\\v2.3.5\\src\\gen\\java\\org\\apache\\hive\\service\\rpc\\thrift\\TI16Column.java"])
prediction = model.predict(Y)
print(prediction)

Y = vectorizer.transform(["spark-master\\examples\\src\\main\\java\\org\\apache\\spark\\examples\\ml\\JavaMaxAbsScalerExample.java"])
prediction = model.predict(Y)
print(prediction)

Y = vectorizer.transform(["spark-master\\examples\\src\\main\\java\\org\\apache\\spark\\examples\\ml\\JavaMinHashLSHExample.java"])
prediction = model.predict(Y)
print(prediction)

Y = vectorizer.transform(["spark-master\\examples\\src\\main\\java\\org\\apache\\spark\\examples\\ml\\JavaMinMaxScalerExample.java"])
prediction = model.predict(Y)
print(prediction)

Prediction
[18]
[258]
[3]
[3]
[3]


In [115]:
# Writing for ifdf

f = open('MoJo_1.2.1/test/test2_b.rsf', 'w')
for element in dir_arr:
    Y = vectorizer.transform([element])
    string = "contain " + str(model.predict(Y)[0]) + " " +str(main_dict[element]) + "\n"
    f.write(string)
    
f.close()

In [34]:
index  = 0
var_array = []
for value in spark_results['variables']:
    var_array.append([index, value])
    #print(index, value)
    index += 1
    
var_df = pd.DataFrame(var_array)
var_df.columns = ['index_val', 'name']
var_df.head()


,index_val,name
0,0,C:\Users\tanji\Desktop\spark-master\common\kvs...
1,1,C:\Users\tanji\Desktop\spark-master\common\kvs...
2,2,C:\Users\tanji\Desktop\spark-master\common\kvs...
3,3,C:\Users\tanji\Desktop\spark-master\common\kvs...
4,4,C:\Users\tanji\Desktop\spark-master\common\kvs...


In [36]:
feature_list = {}
feature_index = 2
for element in spark_results['cells']:
    #print(element)
    try:
        for a in element['values']:
            if a not in feature_list:
                feature_list[a] = feature_index
                feature_index += 1
            #print(a['Call'])
        
        
    except:
        pass
    
feature_list

{'Cast': 2,
 'Call': 3,
 'Return': 4,
 'Use': 5,
 'Create': 6,
 'Parameter': 7,
 'Contain': 8,
 'Import': 9,
 'Extend': 10,
 'Annotation': 11,
 'Implement': 12,
 'Throw': 13}

In [37]:
feature_arr = []
for element in spark_results['cells']:
    #print(array)
    array = [0] * (len(feature_list) + 2)
    values = dict(element['values'])
    #print(element)
    array[0] = element['src']
    array[1] = element['dest']
    for feature in feature_list:
        try:
            value = values[feature]
            array[feature_list[feature]] = value
        except:
            pass
    #print(array)
    feature_arr.append(array)

In [38]:
feature_df = pd.DataFrame(feature_arr)
col_names = ['src', 'dest']
for element in feature_list:
    col_names.append(element)
feature_df.columns = col_names
feature_df.head()

,src,dest,Cast,Call,Return,Use,Create,Parameter,Contain,Import,Extend,Annotation,Implement,Throw
0,796,834,1.0,9.0,1.0,8.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,522,498,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,596,589,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,11,178,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,772,909,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
feature_df['sum'] = feature_df.sum(axis=1) - feature_df['src'] - feature_df['dest']

In [40]:
feature_df.head()

,src,dest,Cast,Call,Return,Use,Create,Parameter,Contain,Import,Extend,Annotation,Implement,Throw,sum
0,796,834,1.0,9.0,1.0,8.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,24.0
1,522,498,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,3.0
2,596,589,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0
3,11,178,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0
4,772,909,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [42]:
G = nx.Graph()
for index, row in feature_df.iterrows():
    G.add_edge(row['src'], row['dest'], weight=row['sum'])
    
len(G.nodes)

979

In [43]:
adj_mat = nx.adjacency_matrix(G)

In [52]:
pd.DataFrame(adj_mat.todense()).head()

,0,1,2,3,4,5,6,7,8,9,...,969,970,971,972,973,974,975,976,977,978
0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,4.0,7.0
4,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
### Converts diagonal to max value ###
### Normalizes the entier dataframe ###

adj_mat_df = pd.DataFrame(adj_mat.todense())
np.fill_diagonal(adj_mat_df.values, adj_mat_df.values.max())
x = adj_mat_df.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
adj_mat_df = pd.DataFrame(x_scaled)
print(adj_mat_df.shape)
adj_mat_df.head()

(979, 979)


,0,1,2,3,4,5,6,7,8,9,...,969,970,971,972,973,974,975,976,977,978
0,1.000000,0.115385,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
1,0.115385,1.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
2,0.000000,0.000000,1.000000,0.014423,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
3,0.000000,0.000000,0.014423,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.024038,0.024038,0.0,0.0,0.0,0.0,0.019231,0.033654
4,0.000000,0.000000,0.000000,0.000000,1.0,0.009615,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000


In [57]:
input_n_clusters = 300
main_cluster_arr = []
col_counter = 0

print(input_n_clusters)
cluster = AgglomerativeClustering(n_clusters=input_n_clusters, affinity='euclidean', linkage='ward')
cluster_result = cluster.fit_predict(adj_mat_df.values)

cluster_result_arr = []
cluster_counter = 0
for element in cluster_result:
    #print(element)
    cluster_result_arr.append(element)

main_cluster_arr.append(cluster_result_arr)
input_n_clusters = input_n_clusters // 2
col_counter += 1

300


In [116]:
f = open('MoJo_1.2.1/test/test2_a.rsf', 'w')
for element in cluster_result:
    #print(element)
    string = "contain " + str(element) + " " + str(main_dict[main_dict_val[element]]) + "\n"
    f.write(string)
    
    
for i in range(len(dir_arr)-len(G.nodes)):
    tbc = (i+len(G.nodes))
    string = "contain " + str(tbc) + " " + str(tbc) + "\n"
    f.write(string)
    
f.close()

## Below are for hypertuning

In [167]:
n_clusters = [50,100,150,200,250,300]
affinity = ['euclidean', 'l1', 'l2', 'manhattan', 'cosine']
linkage = ['complete', 'average', 'single']


### Uncomment below for testing purposes ###
#n_clusters = [50]
#affinity = ['euclidean']
#linkage = ['complete']


def alvin_god(user_n_cluster, user_affinity, user_linkage):
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(dir_arr)

    true_k = user_n_cluster
    model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
    model.fit(X)

    #print("Top terms per cluster:")
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    
    filename1 = 'MoJo_1.2.1/test/' + str(user_n_cluster) + '_' + str(user_affinity) + "_" + str(user_linkage) + '_b' +".rsf"
    f = open(filename1, 'w')
    for element in dir_arr:
        Y = vectorizer.transform([element])
        string = "contain " + str(model.predict(Y)[0]) + " " +str(main_dict[element]) + "\n"
        f.write(string)
    
    f.close()
    
    
    
    main_cluster_arr = []
    col_counter = 0

    
    cluster = AgglomerativeClustering(n_clusters=user_n_cluster, affinity=user_affinity, linkage=user_linkage)
    cluster_result = cluster.fit_predict(adj_mat_df.values)

    cluster_result_arr = []
    cluster_counter = 0
    for element in cluster_result:
        #print(element)
        cluster_result_arr.append(element)

    main_cluster_arr.append(cluster_result_arr)
    
    col_counter += 1
    
    filename2 = 'MoJo_1.2.1/test/' + str(user_n_cluster) + '_' + str(user_affinity) + "_" + str(user_linkage) + '_a' +".rsf"
    f = open(filename2, 'w')
    for element in cluster_result:
        #print(element)
        string = "contain " + str(element) + " " + str(main_dict[main_dict_val[element]]) + "\n"
        f.write(string)


    for i in range(len(dir_arr)-len(G.nodes)):
        tbc = (i+len(G.nodes))
        string = "contain " + str(tbc) + " " + str(tbc) + "\n"
        f.write(string)

    f.close()
    
    return filename1, filename2
    

In [168]:
filename_arr = []

for number in n_clusters:
    for affinity_type in affinity:
        for linkage_type in linkage :
            filename_1, filename_2 = alvin_god(number, affinity_type, linkage_type)
            filename_1 = filename_1.replace('MoJo_1.2.1/', '')
            filename_2 = filename_2.replace('MoJo_1.2.1/', '')
            filename_arr.append((filename_1, filename_2))
            print(filename_1, filename2)

test/50_euclidean_complete_b.rsf MoJo_1.2.1/test/50_euclidean_complete_a.rsf
test/50_euclidean_average_b.rsf MoJo_1.2.1/test/50_euclidean_complete_a.rsf
test/50_euclidean_single_b.rsf MoJo_1.2.1/test/50_euclidean_complete_a.rsf
test/50_l1_complete_b.rsf MoJo_1.2.1/test/50_euclidean_complete_a.rsf
test/50_l1_average_b.rsf MoJo_1.2.1/test/50_euclidean_complete_a.rsf
test/50_l1_single_b.rsf MoJo_1.2.1/test/50_euclidean_complete_a.rsf
test/50_l2_complete_b.rsf MoJo_1.2.1/test/50_euclidean_complete_a.rsf
test/50_l2_average_b.rsf MoJo_1.2.1/test/50_euclidean_complete_a.rsf
test/50_l2_single_b.rsf MoJo_1.2.1/test/50_euclidean_complete_a.rsf
test/50_manhattan_complete_b.rsf MoJo_1.2.1/test/50_euclidean_complete_a.rsf
test/50_manhattan_average_b.rsf MoJo_1.2.1/test/50_euclidean_complete_a.rsf
test/50_manhattan_single_b.rsf MoJo_1.2.1/test/50_euclidean_complete_a.rsf
test/50_cosine_complete_b.rsf MoJo_1.2.1/test/50_euclidean_complete_a.rsf
test/50_cosine_average_b.rsf MoJo_1.2.1/test/50_euclidea

### Before running remeber to copy MoJo into working directory, and then create a test subdirectory :)

# Result collection

In [172]:
import os

for i in range(len(filename_arr)):
    
    command = 'cd C:/Users/tanji/Desktop/FIT4003/MoJo_1.2.1 & ' + 'java MoJo ' + filename_arr[i][1] + ' ' + filename_arr[i][0] + ' >> test/test_results.txt'
    #print(command)
    #os.system('cd C:/Users/tanji/Desktop/FIT4003/MoJo_1.2.1 & java MoJo test/test2_a.rsf test/test2_b.rsf > test/test_results.txt')
    os.system(command)

In [188]:
f = open('MoJo_1.2.1/test/test_results.txt', 'r')

#for line in f:
    #print(line)
final_results = []
for line in f:
    if 'MoJo' in line:
        temp_arr= []
        line = line.replace('MoJo', '')
        line = line.replace('test', '')
        line = line.replace('/', '')
        line = line.replace('(', '')
        line = line.replace(')', '')
        line = line.replace('=', ',')
        line = line.replace('\n', '')
        line = line.split(',')
        #print(line)
        details = line[0].split('_')
        temp_arr.append(details[0])
        temp_arr.append(details[1])
        temp_arr.append(details[2])
        temp_arr.append(line[-1])
        final_results.append(temp_arr)
            
f.close()

In [198]:
final_df_results = pd.DataFrame(final_results)
final_df_results.columns = ['n_clusters', 'affinity', 'linkage', 'n_MoJo']
final_df_results.head()

,n_clusters,affinity,linkage,n_MoJo
0,50,euclidean,complete,160
1,50,euclidean,average,166
2,50,euclidean,single,162
3,50,l1,complete,161
4,50,l1,average,164


In [200]:
final_df_results.to_csv('second_iteration_results.csv', index=False)

In [204]:
final_df_results.sort_values(by='n_MoJo').head()

,n_clusters,affinity,linkage,n_MoJo
7,50,l2,average,158
6,50,l2,complete,159
0,50,euclidean,complete,160
3,50,l1,complete,161
9,50,manhattan,complete,161


In [205]:
final_df_results.sort_values(by='n_MoJo', ascending=False).head()

,n_clusters,affinity,linkage,n_MoJo
88,300,cosine,average,288
78,300,l1,complete,286
89,300,cosine,single,284
83,300,l2,single,284
80,300,l1,single,284
